In [ ]:
import pandas as pd
import h5py as h5
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from maayanlab_bioinformatics.dge import limma_voom_differential_expression
import os

In [ ]:
species = "mouse"
h5_version = "2.2"
gsm4sig_version = 2

# Create a folder for results 
os.makedirs(f'results/{species}/', exist_ok=True)

In [ ]:
# Load ARCHS4 data
f = h5.File(f"../{species}_gene_v{h5_version}.h5", "r")
expression = f['data/expression']
genes = [x.decode('UTF-8') for x in f['meta/genes/symbol']]
samples = [x.decode('UTF-8') for x in f['meta/samples/geo_accession']] #GSMs

In [ ]:
# Define control and pertubration samples
study = 'GSE100363'
ctrl = ['GSM2679560','GSM2679562','GSM2679564','GSM2679566']
pert = ['GSM2679559','GSM2679561','GSM2679563','GSM2679565']

In [ ]:
# Perform DEG analysis with Limma
# study_idx: index of study
# study_name: GSE
# control_samples: list of control GSMs
# pert_samples: list of perturbation GSMs

def degAnalysisLimma (study_idx, study_name, control_samples, pert_samples):

    if (os.path.isfile(f'results/{species}/{study_idx}_{study_name}_limma_results.csv')):
        return
    
    # Find where control and perturbation samples are in ARCHS4
    control_idx = sorted([i for i,x in enumerate(samples) if x in control_samples])
    pert_idx =  sorted([i for i,x in enumerate(samples) if x in pert_samples])
    
    # Subset control samples to dataframe
    control_df = pd.DataFrame(expression[:,control_idx]).astype(int)
    control_df.index = genes
    control_df.columns = ['control' + str(n) for n in range(0,len(control_df.columns))]
    
    # Subset perturbation samples to dataframe
    pert_df = pd.DataFrame(expression[:,pert_idx]).astype(int)
    pert_df.index = genes
    pert_df.columns = ['pert' + str(n) for n in range(0,len(pert_df.columns))]

    # Limma voom expects the raw gene counts
    # Perform DEG analysis
    deg_results = limma_voom_differential_expression(controls_mat= control_df, cases_mat= pert_df, voom_design=True,filter_genes=True)
    
    # Save results to csv file in results folder 
    deg_results.to_csv(f'results/{species}/{study_idx}_{study_name}_limma_results.csv')


In [ ]:
gsm4sig = pd.read_csv(f"data/{species}_gsm4sig_v{gsm4sig_version}.csv", converters = {"ctrl_gsm":eval, "pert_gsm":eval}, index_col=0)

In [ ]:
len(gsm4sig)

In [ ]:
for idx in tqdm(gsm4sig.index.values):
    try:
        degAnalysisLimma(idx, gsm4sig.at[idx, "series_id"], gsm4sig.at[idx, "ctrl_gsm"], gsm4sig.at[idx, "pert_gsm"])
    except Exception as e:
        print(idx)
        print(e)
    

In [ ]:
f.close()

In [ ]:
print(f"Signature statistics for {species}:\n")
print(f"Total Signatures: {len(gsm4sig['extrap_score'])}")
print(f"High quality signatures (score=0,1): \
{sum(gsm4sig['extrap_score'].value_counts()[[0,1]])/sum(gsm4sig['extrap_score'].value_counts())*100:.2f}%")
print(f"Breakdown by extrap_score:\n{gsm4sig['extrap_score'].value_counts()}")
plt.bar(gsm4sig["extrap_score"].value_counts().index, gsm4sig["extrap_score"].value_counts())
plt.show()